In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian
from var_opt import objective_diagonal

import var_opt_lagrange

# Lagrange by Rudy

See Rudy's note on optimizing the variance with Lagrange multiplier method

In [2]:
lih_small = HamiltonianSmall('LiH', 1.5) # this encoding uses reduction techniques
beh2_small = HamiltonianSmall('BeH2', 1.3) # this encoding uses reduction techniques

# 4 qubits
h2_jw_4 = Hamiltonian('H2_STO3g_4qubits', 'jw')
h2_parity_4 = Hamiltonian('H2_STO3g_4qubits', 'parity')
h2_bk_4 = Hamiltonian('H2_STO3g_4qubits', 'bk')

# 8 qubits
h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

In [3]:
hamiltonians = {"lih_small": lih_small,
                "beh2_small": beh2_small,
                "h2_jw_4": h2_jw_4,
                "h2_parity_4": h2_parity_4,
                "h2_bk_4": h2_bk_4,
                "h2_jw": h2_jw,
                "h2_parity": h2_parity,
                "h2_bk": h2_bk}           

An example showing how to call this from `Hamiltonian.PauliRep.local_dists_optimal`

In [4]:
ham = lih_small
energy, state = ham.pauli_rep.ground()

objective = 'diagonal'
method = 'lagrange'

β_optimal_diagonal = ham.pauli_rep.local_dists_optimal(objective, method)

var = ham.pauli_rep.variance_local(energy, state, β_optimal_diagonal)
print(var)

1.5744109284039336


some of this code has been used to build into the function `find_optimal_beta_lagrange` which is called in the above `local_dists_optimal`

In [5]:
def test_iterative_updates(tol=1.0e-5):
    for name, ham in hamiltonians.items():
        
        print("Molecule=", name, "------------------")
        num_qubits = ham.pauli_rep.num_qubits
        energy, state = ham.pauli_rep.ground()
        # compare lagrange method with scipy method
        β_opt = ham.pauli_rep.local_dists_optimal('diagonal', 'scipy')
        # the cost function's value using scipy reference
        objective = objective_diagonal(ham.pauli_rep.dic_tf, β_opt) 
         
        # THIS IS THE EXPERIMENT WITH ITERATIVE UPDATES TO FIND BETAS 
        tol = 1.0e-5
        iter = 0
        β_old = None
        while True and iter < 1000:
            β_new, error = var_opt_lagrange.update_betas(ham.pauli_rep.dic_tf, num_qubits, β_old)
            β_old = β_new
            print("\t", iter, error)
            iter += 1
            if error < tol:
                print("MOLECULE", name, "CONVERGE with error = ", error)
                break

        print("Distance from true values:", var_opt_lagrange.distance(β_new, β_opt)) 

In [6]:
test_iterative_updates()

Molecule= lih_small ------------------
	 0 0.11261586135042559
	 1 0.07825628474170102
	 2 0.05202240430061836
	 3 0.033880031598654124
	 4 0.021925624185318147
	 5 0.014210407724462013
	 6 0.009276649946109662
	 7 0.006139246356234342
	 8 0.004154674175809936
	 9 0.002905510613624632
	 10 0.002119806498677156
	 11 0.0016198954727863284
	 12 0.0012915104421728637
	 13 0.0010641692440765077
	 14 0.000896725360677269
	 15 0.0007662223857614373
	 16 0.0006600844415156557
	 17 0.0005713071286726417
	 18 0.0004957830223325781
	 19 0.00043090791830143344
	 20 0.0003748793822691835
	 21 0.00032634781583358477
	 22 0.00028424134968488756
	 23 0.00024767520732789155
	 24 0.00021590205284284985
	 25 0.00018828245762566505
	 26 0.00016426553446891726
	 27 0.00014337498727634579
	 28 0.0001251982845021868
	 29 0.00010937782368206882
	 30 9.56035007943388e-05
	 31 8.360635727727898e-05
	 32 7.315310258536152e-05
	 33 6.40413728700628e-05
	 34 5.609562006280731e-05
	 35 4.916354569557362e-05
	 36 4.

/opt/anaconda3/envs/qiskit18/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


	 0 0.08601299568057812
	 1 0.06785095529261422
	 2 0.05336477140346753
	 3 0.041895919174394614
	 4 0.0330127509032797
	 5 0.026238967828564923
	 6 0.021095595700498796
	 7 0.01716571559090108
	 8 0.014122504144670376
	 9 0.011726352716024036
	 10 0.00980791700876857
	 11 0.008249025482684047
	 12 0.006966774207784095
	 13 0.0059019694877530116
	 14 0.0050113062598735195
	 15 0.004262255039195648
	 16 0.0036297564160765895
	 17 0.003094073164009261
	 18 0.0026393714790396384
	 19 0.002252761189902437
	 20 0.0019236279237627076
	 21 0.0016431544228918255
	 22 0.0014039673545136815
	 23 0.001199869673944053
	 24 0.0010256330158221704
	 25 0.000876833432031736
	 26 0.0007497192973433829
	 27 0.0006411036876881489
	 28 0.0005482757869589462
	 29 0.0004689273657569065
	 30 0.00040109138261634736
	 31 0.0003430904575660451
	 32 0.0002934934660613148
	 33 0.00025107886519071997
	 34 0.000214803636187312
	 35 0.00018377693523580067
	 36 0.0001572377065691678
	 37 0.00013453564015869415
	 38 0

	 36 6.415588893196285e-05
	 37 5.189972522233232e-05
	 38 4.198875220643859e-05
	 39 3.3973499270855725e-05
	 40 2.7490779743758557e-05
	 41 2.2247099770159188e-05
	 42 1.8005262681414433e-05
	 43 1.457355246038732e-05
	 44 1.1796998931536737e-05
	 45 9.550323352883176e-06
MOLECULE h2_jw CONVERGE with error =  9.550323352883176e-06
Distance from true values: 4.069785344794131e-05
Molecule= h2_parity ------------------
	 0 0.13703045680358492
	 1 0.12176919707590238
	 2 0.10797307480423644
	 3 0.09555901696143272
	 4 0.08458538740888115
	 5 0.07498742012555955
	 6 0.06661545075244509
	 7 0.05929888571566724
	 8 0.05288098612646971
	 9 0.04722918313829185
	 10 0.04223417204372722
	 11 0.037805965254411135
	 12 0.033870021588895585
	 13 0.03036411339351111
	 14 0.02723585727427325
	 15 0.02444075636116801
	 16 0.021940654216782724
	 17 0.019702532072742186
	 18 0.0176975872277526
	 19 0.01590053122900592
	 20 0.014289051847296017
	 21 0.012843392946229075
	 22 0.011546017917217837
	 23 0

KeyboardInterrupt: 